In [7]:
# Use this to reload the module when changes are made
%load_ext autoreload
%autoreload 2
# Enable asyncio in Jupyter
%autoawait asyncio

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
from pathlib import Path

from data.datasets_schema import Datasets

datasets_path = Path("../data/datasets.yaml")
datasets: Datasets = Datasets.from_yaml(datasets_path)
moodle_dataset = datasets["test-0.1.0"]
entries = moodle_dataset.document_storage.entries
# await moodle_dataset.document_storage.download_files("download")
print(len(entries))

1214


In [67]:
from experiments.pdf_scrap import use_pymupdf4llm, use_unstructured

path = entries[
    "[F22] Fundamentals of Computer Architecture/Week 10 - 03 November 2022/Lab 10 (Floating-Point Operations in RISCV).pdf"].path
# path = entries["Week 05 - 29 September 2022/Lecture 5 Slides.pdf"].path
pages = use_unstructured(path)
print(len(pages))

123


In [72]:
import unstructured.documents.elements as el

for p in pages:
    if isinstance(p, el.Title):
        print(f"<h1>{p}</h1>")

<h1>Computer Architecture Lab 10</h1>
<h1>Display of Midterm Works</h1>
<h1>Floating-Point Instructions in RISC-V</h1>
<h1>INNOR OVE sity</h1>
<h1>RISC-V Architecture Layout: Floating-Point Registers</h1>
<h1>Integer Registers</h1>
<h1>RISC-V: 32 Directly Addressable Floating-Point Registers</h1>
<h1>Register Name</h1>
<h1>Symbolic (or ABI)</h1>
<h1>Register Purpose</h1>
<h1>Floating-point temporary registers</h1>
<h1>RISC-V floating-point operands</h1>
<h1>registers double-precision floating-point number.</h1>
<h1>RISC-V floating-point assembly language</h1>
<h1>RISC-V System Calls for External Interaction (just for your reference)</h1>
<h1>Code in reg. a7</h1>
<h1>ON DW BP WN FR</h1>
<h1>Allocate Heap memory</h1>
<h1>Exit program/function Print character Read character</h1>
<h1>Exit program with return value</h1>
<h1>Arguments/Return values</h1>
<h1>Example: Program Using Floating-Point Instructions</h1>
<h1>Sample Program with Floating-Point Instructions</h1>
<h1>const32: .float 32.

In [258]:
import ollama
from langchain_community.embeddings import OllamaEmbeddings

ollama.pull("mxbai-embed-large")  # https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1
ollama_embeddings = OllamaEmbeddings(model="mxbai-embed-large",
                                     embed_instruction="Represent this sentence for searching relevant passages: ",
                                     query_instruction="Represent this sentence for searching relevant passages: ")

In [250]:
from inspect import cleandoc
from langchain_core.prompts import SystemMessagePromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = Ollama(model="llama3:instruct")

prompt_template = cleandoc("""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an assistant for generating dataset for a evaluating document semantic search engine.
Use the following content to generate exactly one query for the given content. Content may contain schemas, if it messy, just ignore it.
Content: {content}

Return only query, do not return any other information, comments.
Try your best to generate human-like queries. Sometimes, people will just ask keywords or phrases, may be summary or paraphrase of the content, or may be a question or task from their test or assignment. Do not include "What is", "How do I", and "Can you" in the query. Also query may contain mistakes or may not be perfect, but it should be relevant to the content.
<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
Here's a query generated from the given content:
""")

prompt = ChatPromptTemplate(
    messages=[SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            template=prompt_template,
            input_variables=["content"]
        )
    )],
    input_variables=["content"]
)
generate_question_chain = ({"content": RunnablePassthrough()} |
                           prompt |
                           llm |
                           StrOutputParser())



In [259]:
def task(path: Path):
    """
    Open a PDF file and query llm to generate exactly one question for the content of the document 
    """

    pages = use_pymupdf4llm(path)

    questions = []
    i = 0
    for page in pages:
        i += 1
        content = page["text"]
        question = generate_question_chain.invoke(content)
        question = question.strip("\"")
        questions.append((page, question))
        print(f"> Question {i}: {question}")

    return questions

In [256]:
import json

for _, entry in entries.items():
    path = entry.path
    if path is None:
        continue
    if path.suffix != ".pdf":
        continue
    print(f"Processing {path}")
    questions = task(path)
    dest = path.with_suffix(".questions.json")

    with open(dest, "w") as f:
        json.dump(questions, f, indent=4)

Processing ../scripts/output/Week 05 - 29 September 2022/Lab 05 Slides.pdf
Question 1: Arithmetic operations combinational logic circuits adders

Question 2: How to sum binary values like 0011 and 0010?

Question 3: Sum binary values 4-bit each

Question 4: Binary addition example 0011 0010 result

Question 5: Difference between Half Adder Circuit and Full Adder Circuit

Question 6: XOR gates for 4-bit input

Question 7: Implement 4-bit adder circuit with switches for inputs and LEDs for output

Question 8: Half subtractor circuit comparison with half adder

Question 9: Difference between half subtractor and half adder circuits

Question 10: Circuits for subtraction

Question 11: 4-bit input arguments circuit generalization XOR NOT AND OR 0 1

Processing ../scripts/output/Week 05 - 29 September 2022/Lecture 5 Slides.pdf
Question 1: Lecture notes on computer architecture number systems

Question 2: System for converting numbers between binary and floating point

Question 3: Types of num

KeyboardInterrupt: 